In [ ]:
#scrapping wit selenium
import os

os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'
    
os.environ['PATH']

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver_options = Options()
driver_options.headless = True
browser = webdriver.Firefox(options=driver_options)
driver = browser

In [ ]:
driver.get('web-url')

#get images
imgs = driver.find_elements_by_class_name('product-photo-img')
for img in imgs[:4]:
    images_url_list.append(img.get_attribute('src'))
for img in imgs[4:]:   
    images_url_list.append( img.get_attribute('data-src'))
    
#get_titles
brands = driver.find_elements_by_class_name('product-title')
for brand in brands:   

    brand_list.append(brand.text)
#get_titles
shoes = driver.find_elements_by_class_name('product-series')
for shoe in shoes:   
    shoe_list.append(shoe.text)
#get_price
prices = driver.find_elements_by_class_name('price-info')
for price in prices:   
    price_list.append(price.text)

In [ ]:
#download images from dataset url 
import os
import requests
from pathlib import Path
import re
errors_list = []

def get_img_download(index,brand,shoe,url: str, dest_folder: str):
    brand = re.sub('[^a-zA-Z0-9\n\.]', '_', brand).replace(" ","_").lower() 
    shoe = re.sub('[^a-zA-Z0-9\n\.]', '_', shoe).replace(" ","_").lower()
    numb = index
    hhd = dest_folder
    dest_folder = Path(f"{hhd}").mkdir(parents=True, exist_ok=True)

    filename = str(index)+"_"+str(brand)+"_"+str(shoe) # be careful with file names
    file_path = Path(f"{hhd}/{filename}.jpg")
    df['des_f'][numb] = (f"{hhd}/{filename}.jpg")
    r = requests.get(url, stream=True)
    if r.ok:
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 8):
                if chunk:
                    f.write(chunk)
                    f.flush()
                    os.fsync(f.fileno())
    else:  # HTTP status code 4XX/5XX
        errors_list.append(numb)
        print("Download failed: status code {}\n{}".format(r.status_code, r.text))
        

url_to_save = '/home/migue/Documentos/final_project/notebook/data/lastdb'
df.apply(lambda x: get_img_download( x['index_num'],x['brand'],x['shoe'],x['img_url'],url_to_save), axis=1 )

In [ ]:
#crop images

def cropp_images(x):
    # Size of the image in pixels (size of orginal image) 
    # (This is not mandatory) 
    print(x)
    img_true = os.path.exists(x)
    
    if img_true:
        immm = Image.open(x).convert('RGB') 
        width, height = immm.size 
        b = x.split("/")
        name_img = b[-1]
        # Setting the points for cropped image 
        new_width = 280
        new_height = 200
        left = (width - new_width)/2
        top = (height - new_height)/2
        right = (width + new_width)/2
        bottom = (height + new_height)/2


        # Cropped image of above dimension 
        # (It will not change orginal image) 
        imm1 = immm.crop((left, top, right, bottom)) 

        # Shows the image in image viewer 
        imm1.save(f'data/images/{name_img}', optimize=True)
    else:
        print(x)